## **Wandb Link:** 

https://wandb.ai/sxs200126/CIFAR10_CNN/reports/Shishir_hw5--VmlldzoxMjI0NTEy?accessToken=quhmu5ctphel12f82j8k7d61zdz06xvonxdt8l1frfaahwn4g3hpfe9pmzomblfo

## **Question 2 (15 Points):** 
In this question, you can experiment with different convent architectures on CIFAR-10. You can experiment with different architectures, hyper-parameters, loss functions, and optimizers to train a model that achieves close to 80% accuracy on the CIFAR-10 validation set within 10 epochs


In [1]:
%%capture
!pip install wandb --upgrade

In [2]:
# Import wandb
import wandb

# Login to W&B
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
pip install torch-lr-finder

In [4]:
# Importing the necessary libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchsummary import summary

from torch.optim.lr_scheduler import ReduceLROnPlateau, ExponentialLR, CyclicLR, OneCycleLR, StepLR
from torch_lr_finder import LRFinder

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import random

from datetime import datetime
from pathlib import Path
import plotly.io as pio
pio.renderers.default = 'colab'

In [ ]:
# Import random function
import random

# Fix seed value
SEED = 2345
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_folder = Path('/content/drive/MyDrive/Data/DL')

In [ ]:
lecture_folder = Path('/content/drive/MyDrive/Deep Learning/HW5')

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

## **Data Download and Transform to tensor**

In [ ]:
# Transform to convert images to pytorch tensors and normalize the data
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4655), (0.3530,0.1994,0.2010))])
train_full = torchvision.datasets.CIFAR10(root=data_folder,
                                              train=True, 
                                              transform=trans,
                                              download=True)
trainset, validset = torch.utils.data.random_split(train_full, [40000, 10000], generator=torch.Generator().manual_seed(42) )
testset  = torchvision.datasets.CIFAR10(root=data_folder,
                                              train=False, 
                                              transform=trans,
                                              download=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
classes = train_full.classes
classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [ ]:
class_count = {}
for _, index in train_full:
    label = classes[index]
    if label not in class_count:
        class_count[label] = 0
    class_count[label] += 1
class_count

{'airplane': 5000,
 'automobile': 5000,
 'bird': 5000,
 'cat': 5000,
 'deer': 5000,
 'dog': 5000,
 'frog': 5000,
 'horse': 5000,
 'ship': 5000,
 'truck': 5000}

In [ ]:
train_full.data.shape

(50000, 32, 32, 3)

In [ ]:
testset.data.shape

(10000, 32, 32, 3)

### Check Transformation

- Check if transformation are working correctly.
- The transformations are applied at the time of calling dataloader

In [ ]:
check_loader = torch.utils.data.DataLoader(trainset, batch_size = 32, shuffle = True)

In [ ]:
# check number of batches
len(check_loader)

1250

In [ ]:
# check total training examples
len(check_loader.dataset)

40000

In [ ]:
# check imputs and outputs 
for input, target in check_loader:
  print(f'shape of inputs is :{input.shape}')
  print(f'\nmax input value  :{input.max()}')
  print(f'\nmin input value  :{input.min()}')
  print(f'\nmean input value  :{input.mean()}')
  print(f'\nstd input value  :{input.std()}')
  print(f'\nshape of targets is :{target.shape}')
   
  break

shape of inputs is :torch.Size([32, 3, 32, 32])

max input value  :2.6592040061950684

min input value  :-2.418254852294922

mean input value  :-0.08662816882133484

std input value  :1.1399555206298828

shape of targets is :torch.Size([32])


First let's define labels for our dataset as dataset contains numerical values for now.

## Get Labels

In [ ]:
def get_CIFAR10_labels(labels):  
    """ 
    Function to generate labels.
    Input: numerical labels
    Output: actual string labels
    """

    # Create a list of labels
    text_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                   'dog', 'frog', 'horse', 'ship', 'truck']

    # Return text_labels according to numerical values
    return [text_labels[int(i)] for i in labels]

## **Small Dataset**

In [ ]:
# n sample points
train_sample_size = int(len(trainset)/10)
valid_sample_size = int(len(validset)/10)

# Getting n random indices
train_subset_indices = random.sample(range(0, len(trainset)), train_sample_size)
valid_subset_indices = random.sample(range(0, len(testset)), valid_sample_size)

# Getting subset of dataset
train_subset = torch.utils.data.Subset(trainset, train_subset_indices)
valid_subset = torch.utils.data.Subset(validset, valid_subset_indices)

# Model CNN

In [ ]:
class CIFAR10CNN(nn.Module):
    
    def __init__(self):

      super().__init__()

      super(CIFAR10CNN, self).__init__()
      

      
      self.conv1_layer = nn.Sequential(
          nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding='same'), # 32 * 32
          nn.ReLU(),
          nn.BatchNorm2d(32),
          nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding='same'), # 32 * 32
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride = 2 ) # 16 * 16
      )

      self.conv2_layer = nn.Sequential(
          nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding='same'), # 16 * 16
          nn.ReLU(),
          nn.BatchNorm2d(64),
          nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding='same'), # 16 * 16
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2) # 8 * 8
      )

      self.conv3_layer = nn.Sequential(
          nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding='same'), # 8 * 8
          nn.ReLU(),
          nn.BatchNorm2d(128),
          nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding='same'), ## 8 * 8
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2) # 4 * 4
      )
      

     
      self.flatten = nn.Flatten()
      
      self.fc1 = nn.Linear(4*4*128, out_features=128)
      
      self.fc2 = nn.Linear(128, 10)

      #self.fc3 = nn.Linear(512, 10)

      self.drop1 = nn.Dropout(0.2)
      self.drop2 = nn.Dropout(0.2)
      
      
      
        
    def forward(self, x):
        # conv layers
        out = self.conv1_layer(x)
        out = self.drop1(self.conv2_layer(out))
        out = self.drop2(self.conv3_layer(out))
        #out = self.conv4_layer(out)

        # flatten befrore input to linear layer
        out = self.flatten(out)

        # linear hidden layers
        out = F.relu(self.fc1(out))
        #out = self.drop1(out)
        #out = F.relu(self.fc2(out))
        #out = self.drop2(out)

        # output layer - no softmax as it is applied by nn.CrossEntropyLoss

        out = self.fc2(out)
        
        return out

In [ ]:
summary(CIFAR10CNN().cuda(), (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
              ReLU-2           [-1, 32, 32, 32]               0
       BatchNorm2d-3           [-1, 32, 32, 32]              64
            Conv2d-4           [-1, 32, 32, 32]           9,248
              ReLU-5           [-1, 32, 32, 32]               0
         MaxPool2d-6           [-1, 32, 16, 16]               0
            Conv2d-7           [-1, 64, 16, 16]          18,496
              ReLU-8           [-1, 64, 16, 16]               0
       BatchNorm2d-9           [-1, 64, 16, 16]             128
           Conv2d-10           [-1, 64, 16, 16]          36,928
             ReLU-11           [-1, 64, 16, 16]               0
        MaxPool2d-12             [-1, 64, 8, 8]               0
          Dropout-13             [-1, 64, 8, 8]               0
           Conv2d-14            [-1, 12

# Training Functions

## Training Epoch 

In [ ]:
def train(train_loader, model, optimizer, loss_function, log_batch, log_interval, grad_clipping, max_norm):

  """ 
  Function for training the model in each epoch
  Input: iterator for train dataset, initial weights and bias, epochs, learning rate.
  Output: final weights, bias, train loss, train accuracy
  """
  # initilalize variables as global
  # these counts will be updated every epoch
  global example_ct_train
  global batch_ct_train

  # Training Loop loop
  # Initialize train_loss at the he start of the epoch
  running_train_loss = 0
  running_train_correct = 0
  
  # put the model in training mode
  model.train()

  # Iterate on batches from the dataset using train_loader
  for input, targets in train_loader:
    
    # move inputs and outputs to GPUs
    input = input.to(device)
    targets = targets.to(device)

    # Forward pass
    output = model(input)
    loss = loss_function(output, targets)

    # Correct prediction
    y_pred = torch.argmax(output, dim = 1)
    correct = torch.sum(y_pred == targets)

    example_ct_train +=  len(targets)
    batch_ct_train += 1

    # set gradients to zero 
    optimizer.zero_grad()

    # Backward pass
    loss.backward()

    # Gradient Clipping
    if grad_clipping:
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm, norm_type=2)

    # Update parameters using their gradient
    optimizer.step()
          
    # Add train loss of a batch 
    running_train_loss += loss.item()

    # Add Corect counts of a batch
    running_train_correct += correct

    # log batch loss and accuracy
    if log_batch:
      if ((batch_ct_train + 1) % log_interval) == 0:
        wandb.log({f"Train Batch Loss  :": loss})
        wandb.log({f"Train Batch Acc :": correct/len(targets)})

    #scheduler.step()
  # Calculate mean train loss for the whole dataset for a particular epoch
  train_loss = running_train_loss/len(train_loader)



  # Calculate accuracy for the whole dataset for a particular epoch
  train_acc = running_train_correct/len(train_loader.dataset)

  return train_loss, train_acc

## Validation/Test Epoch

In [ ]:
def valid(loader, model, optimizer, loss_function, log_batch, log_interval):

  """ 
  Function for training the model and plotting the graph for train & valid loss vs epoch.
  Input: iterator for train dataset, initial weights and bias, epochs, learning rate, batch size.
  Output: final weights, bias and train loss and valid loss for each epoch.
  """

  # initilalize variables as global
  # these counts will be updated every epoch
  global example_ct_valid
  global batch_ct_valid

  # Validation loop
  # Initialize train_loss at the he strat of the epoch
  running_valid_loss = 0
  running_valid_correct = 0
  
  # put the model in evaluation mode
  model.eval()

  with torch.no_grad():
    for input,targets in loader:

      # move inputs and outputs to GPUs
      input = input.to(device)
      targets = targets.to(device)

      # Forward pass
      output = model(input)
      loss = loss_function(output,targets)

      # Correct Predictions
      y_pred = torch.argmax(output, dim = 1)
      correct = torch.sum(y_pred == targets)

      # count of images and batches
      example_ct_valid +=  len(targets)
      batch_ct_valid += 1

      # Add valid loss of a batch 
      running_valid_loss += loss.item()

      # Add correct count for each batch
      running_valid_correct += correct

      # log batch loss and accuracy
      if log_batch:
        if ((batch_ct_valid + 1) % log_interval) == 0:
          wandb.log({f"Valid Batch Loss  :": loss})
          wandb.log({f"Valid Batch Accuracy :": correct/len(targets)})


    # Calculate mean valid loss for the whole dataset for a particular epoch
    valid_loss = running_valid_loss/len(valid_loader)

    # scheduler step
    #scheduler.step(valid_loss)
    # scheduler.step()

    # Calculate accuracy for the whole dataset for a particular epoch
    valid_acc = running_valid_correct/len(valid_loader.dataset)
    
  return valid_loss, valid_acc

##  Model Training Loop


In [ ]:
def train_loop(train_loader, valid_loader, model, loss_function, optimizer, epochs, device, patience, early_stopping,
               file_model):

  '''
  model: specify your model for training
  criterion: loss function 
  optimizer: optimizer like SGD , ADAM etc.
  train loader: function to carete batches for training data
  loader : function to create batches for valid data set
  file_model : specify file name for saving your model. This way we can upload the model weights from file. We will not to run model again.
  

  '''
  # Create lists to store train and valid loss at each epoch

  train_loss_history = []
  valid_loss_history = []
  train_acc_history = []
  valid_acc_history = []
  delta = 0
  best_score = None
  valid_loss_min = np.Inf
  counter_early_stop=0
  early_stop=False


  # Iterate for the given number of epochs
  for epoch in range(epochs):
    t0 = datetime.now()
    # Get train loss and accuracy for one epoch

    train_loss, train_acc = train(train_loader, model, optimizer, loss_function, 
                                  wandb.config.log_batch, wandb.config.log_interval,
                                  wandb.config.grad_clipping, wandb.config.max_norm)
    valid_loss, valid_acc = valid(valid_loader, model, optimizer, loss_function,
                                    wandb.config.log_batch, wandb.config.log_interval)

    dt = datetime.now() - t0

    # Save history of the Losses and accuracy
    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)
    valid_loss_history.append(valid_loss)
    valid_acc_history.append(valid_acc)

    if early_stopping:
      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        counter_early_stop += 1
        print(f'Early stoping counter: {counter_early_stop} out of {patience}')
        if counter_early_stop > patience:
          early_stop = True

      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        counter_early_stop=0
        valid_loss_min = valid_loss

      if early_stop:
        print('Early Stopping')
        break

    else:

      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        print(f'Validation loss has not decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Not Saving Model...')
      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss



    # Log the train and valid loss to W&B
    wandb.log({f"Train epoch Loss :": train_loss, f"Valid epoch Loss :": valid_loss })
    wandb.log({f"Train epoch Acc :": train_acc, f"Valid epoch Acc :": valid_acc})



    # Print the train loss and accuracy for given number of epochs, batch size and number of samples
    print(f'Epoch : {epoch+1} / {epochs}')
    print(f'Time to complete {epoch+1} is {dt}')
    # print(f'Learning rate: {scheduler.get_last_lr()}')
    #print(f'Learning rate: {scheduler._last_lr[0]}')
    print(f'Train Loss: {train_loss : .4f} | Train Accuracy: {train_acc * 100 : .4f}%')
    print(f'Valid Loss: {valid_loss : .4f} | Valid Accuracy: {valid_acc * 100 : .4f}%')
    print()
    torch.cuda.empty_cache()

  return train_loss_history, train_acc_history, valid_loss_history, valid_acc_history


#Model Training

## **Meta data**



In [ ]:
hyperparameters = dict(
    epochs = 10,
    output_dim = 10, 
    
    batch_size = 64,
    learning_rate = 0.01,
    dataset="CIFAR10",
    architecture="CNN",
    log_interval = 25,
    log_batch = True,
    file_model = lecture_folder/'CIFAR10_CNN_experiment_48.pt',
    grad_clipping = True,
    max_norm = 1,
    patience = 5,
    early_stopping = True,
    weight_decay = 0,
    scheduler_factor = 0,
    scheduler_patience = 0,
   )

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Initialize wandb

In [ ]:
wandb.init(name = 'CIFAR10CNN_hw5_exp_48', project = 'CIFAR10_CNN', config = hyperparameters)

Train Batch Acc :,▁▂▁▁▃▃▄▅▅▅▆█▅▇▅▇▆▆▇▆█▇▆█▇▆▇▇█▇█████████▇
Train Batch Loss :,██▇▇▅▆▆▅▅▃▃▃▄▄▄▂▃▃▂▃▂▂▃▂▂▄▃▃▁▂▂▂▁▁▂▂▁▁▂▂
Train epoch Acc :,▁▃▅▆▆▇▇███
Train epoch Loss :,█▅▄▃▂▂▂▁▁▁
Valid Batch Accuracy :,▂▂▂▁▆▂▃▃▆▆▅▃▃▅▆▅█▆▇▇▇▇█▇█▅▆▇▇▇▆█▇▇▇▇▇▆▆▇
Valid Batch Loss :,█▇▇▇▆▇▇▆▅▃▅▆▆▄▃▅▂▄▂▂▂▃▂▂▃▃▄▃▂▃▂▂▄▂▁▃▂▃▄▂
Valid epoch Acc :,▁▄▅▆▇▇▇█▇█
Valid epoch Loss :,█▆▄▄▂▂▂▂▁▁
Train Batch Acc :,0.625
Train Batch Loss :,0.9916
Train epoch Acc :,0.70307


In [ ]:
wandb.config.device = device
print(wandb.config.device )

cuda:0


## Specify Dataloader, Loss_function, Model, Optimizer, Weight Initialization

In [ ]:
# Fix seed value
SEED = 2345
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Data Loader
train_loader = torch.utils.data.DataLoader(trainset, batch_size=wandb.config.batch_size, shuffle = True)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=wandb.config.batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(testset, batch_size=wandb.config.batch_size,   shuffle = False)

# cross entropy loss function
loss_function = nn.CrossEntropyLoss()

# device 
model = CIFAR10CNN()

def init_weights(m):
  if type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        torch.nn.init.zeros_(m.bias)

  if type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        torch.nn.init.zeros_(m.bias)

        
# apply initialization recursively  to all modules
# model.apply(init_weights)

wandb.config.init_weights = init_weights

# put model to GPUs
model.to(wandb.config.device)

# Intialize stochiastic gradient descent optimizer
optimizer = torch.optim.SGD(model.parameters(), lr = wandb.config.learning_rate, weight_decay=wandb.config.weight_decay, momentum = 0.9)
#optimizer = torch.optim.Adam(model.parameters(), lr = wandb.config.learning_rate, weight_decay=wandb.config.weight_decay)

wandb.config.optimizer = optimizer

#scheduler = ReduceLROnPlateau(optimizer, mode='min', factor= wandb.config.scheduler_factor, 
 #                             patience=wandb.config.scheduler_patience, verbose=True)

#scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, total_steps=len(train_loader) * 10 , epochs=10, three_phase=True)

#scheduler = StepLR(optimizer, gamma=0.4,step_size=1, verbose=True)

## Train Model and Save best model

In [ ]:
wandb.watch(model, log = 'all', log_freq=25, log_graph=True)

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [ ]:
example_ct_train, batch_ct_train, example_ct_valid, batch_ct_valid = 0, 0, 0, 0
train_loss_history, train_acc_history, valid_loss_history, valid_acc_history = train_loop(train_loader, valid_loader, model, loss_function, optimizer, 
                                                                                          wandb.config.epochs, wandb.config.device,
                                                                                          wandb.config.patience, wandb.config.early_stopping,
                                                                                          wandb.config.file_model)

Validation loss has decreased (inf --> 1.166863). Saving Model...
Epoch : 1 / 10
Time to complete 1 is 0:00:20.788466
Train Loss:  1.4333 | Train Accuracy:  47.2575%
Valid Loss:  1.1669 | Valid Accuracy:  59.2100%

Validation loss has decreased (1.166863 --> 0.895109). Saving model...
Epoch : 2 / 10
Time to complete 2 is 0:00:20.770825
Train Loss:  0.9970 | Train Accuracy:  64.3350%
Valid Loss:  0.8951 | Valid Accuracy:  68.0700%

Validation loss has decreased (0.895109 --> 0.798625). Saving model...
Epoch : 3 / 10
Time to complete 3 is 0:00:20.823256
Train Loss:  0.8336 | Train Accuracy:  70.7275%
Valid Loss:  0.7986 | Valid Accuracy:  71.8300%

Validation loss has decreased (0.798625 --> 0.711801). Saving model...
Epoch : 4 / 10
Time to complete 4 is 0:00:20.808625
Train Loss:  0.7352 | Train Accuracy:  73.8000%
Valid Loss:  0.7118 | Valid Accuracy:  74.9000%

Validation loss has decreased (0.711801 --> 0.648964). Saving model...
Epoch : 5 / 10
Time to complete 5 is 0:00:20.719397
Tr

# **Accuracy and Predictions**

Now we have final values for weights and bias after training the model. We will use these values to make predictions on the test dataset.

## Function to get predictions

In [ ]:
def get_acc_pred(data_loader, model):
  """ 
  Function to get predictions for a given test set and calculate accuracy.
  Input: Iterator to the test set.
  Output: Prections and Accuracy for test set.
  """
  with torch.no_grad():
    # Array to store predicted labels
    predictions = torch.Tensor()
    predictions = predictions.to(device)

    # Array to store actual labels
    y = torch.Tensor()
    y = y.to(device)
    # Iterate over batches from test set
    for input, targets in data_loader:
      
      # move inputs and outputs to GPUs
      input = input.to(device)
      targets = targets.to(device)

      # Calculated the predicted labels
      output = model(input)

      # Choose the label with maximum probability
      indices = torch.argmax(output, dim = 1)

      # Add the predicted labels to the array
      predictions = torch.cat((predictions, indices)) 

      # Add the actual labels to the array
      y = torch.cat((y, targets)) 

    # Check for complete dataset if actual and predicted labels are same or not
    # Calculate accuracy
    acc = (predictions == y).float().mean()

  # Return array containing predictions and accuracy
  return predictions, acc
  

## Load saved model from file 

In [ ]:
model_CNN =  CIFAR10CNN()
model_CNN.to(device)
model_CNN.load_state_dict(torch.load(wandb.config.file_model))

<All keys matched successfully>

In [ ]:
print(wandb.config.file_model)

/content/drive/MyDrive/Deep Learning/HW5/CIFAR10_CNN_experiment_48.pt


In [ ]:
# Get the prediction and accuracy for the test dataset
predictions, acc_test = get_acc_pred(test_loader, model_CNN)

In [ ]:
# Print Accuracy for test dataset
print(acc_test * 100)
wandb.config.test_accuracy = acc_test

tensor(78.3500, device='cuda:0')
